# Data Wrangling and Cleaning

In [17]:
#Here are the packages I'll be using:
import pandas as pd
import spacy
import numpy as np

#load in spacy nlp stuff
nlp = spacy.load("en_core_web_lg")
df_SUBTlog10 = pd.read_csv('data/word_metrics/SUBTLEXus_log10.csv') 


In [18]:
from pathlib import Path
import os
import glob

#create a path object that we can loop through to access the text files
reading_dir = Path('passages') #using Jupyter Lab - default dir is folder code is in

#create a list of reading files to be looped through later
reading_files = list(reading_dir.glob('*.txt'))

print(reading_files) #Just checking the output

#os.listdir()

[WindowsPath('passages/air.txt'), WindowsPath('passages/mouse_in_the_house.txt'), WindowsPath('passages/the_brain_and_five_senses.txt'), WindowsPath('passages/the_surprise.txt'), WindowsPath('passages/toads.txt')]


In [19]:
text = reading_files[0].read_text(encoding='utf-8') #making sure that path object works as it should
print(text) #Checking that there are no strange formatting errors within text.

nlp_text = nlp(text)

Air is all around us. But we can't see it. How do we know it is there? There are many ways. We can see what air does. Moving air is called wind. Wind moves plants. Wind moves dirt. Strong winds can move heavy things. Strong winds can even move a house. We can weigh air. We can weigh two balloons. The one with a lot of air weighs more. We can see what air does. We can weigh air. Then we know it is there. 


In [20]:
######################################
## Creating a DataFrame that
## has all passages tokenized
## and associated with a parts-of-speech tag
######################################

pos_tuple = [] #This will be used to create a df, and will contain a tuple of all variables of interest

for i, f in enumerate(reading_files):
    sent_num = 0
    print(f.name) #Print which file we're working on
    text = f.read_text(encoding='utf-8') #read the files and save the texts
    text = text.lower() #Lowercase all of the text
    nw = 0 #This keeps track of the order of words within the passage
    #Let's go ahead and tokenize the text
    punct_text = nlp(text) #Tokenize text
    filename = f.name #Read filename
    
    tokenized_text = [] #just a list of all of the words that have been nlp'd, with both punct and spaces filtered out
    first_cont_found = 0 #I want this to reset everytime a new passage is analyzed -- a check to see if the first content word has been found
    
    for word in punct_text:#loop through the tokenized text and get rid of all punctuation and spaces
        if not word.is_punct and not word.pos_ == "SPACE":
            tokenized_text.append(word)
    
    #The code below calculates the "paired semantic-similarity" score, or the semantic-similarity score between a target-word and the most-recent content word.
    for i in range(len(tokenized_text)): #Loop through list of tokenized words
        if not tokenized_text[i].is_stop: 
            if first_cont_found == 0: #If current word is NOT a stop word, and the first stop word has not been found yet
                prior_word = tokenized_text[i] #This lets us get our first "prior" content word -- essentially, this will be our "primer" word
                first_cont_found = 1 #This stops this loop from occuring again for this passage once the first content word is found
                sing_prior_sim = 0 #Since this first content word can't be compared to anything, 0 is the "paired semantic-similarity" score
                print("first content word found:", prior_word, first_cont_found)
            elif first_cont_found == 1: #Now, for all following non-stop words, I can just do this:
                curr_loc = i #This will increase by one as I loop through each word within the passage, and keeps track of which word is the current "target" word
                curr_word = tokenized_text[i] #This assigns the current "target" word to variable curr_word
                sing_prior_sim = curr_word.similarity(prior_word) #This calculates the semantic similarity between the "target" word (curr_word) and "primer" word (prior_word)
                #print("content words can be compared:",curr_word, prior_word, first_func_found, loc_dist)
                prior_word = curr_word #Now the "target" word will become the "primer" word for the next content word in the passage
        else: 
            #If none are true -- current word IS a stop word
            sing_prior_sim = 0 #assign it to be 0 -- this will identify this word as a stop word or a content word in which a paired semantic-similarity score was impossible to calculate.
            #All words with a value of 0 for this variable will be filtered out during analysis.
        
        if tokenized_text[i].is_sent_start: #Only triggers on the first word of a sentence
            word_list = [] #This essentially stores all words from the current sentence
            #And, it also resets when a new sentence is detected.
            sent_num += 1 #This keeps track of which sentence we're currently in.
            m = 0 #This is later used to keep track of which word in the sentence we are analyzing as the "target" word. It is here, because we want it to reset at the start of every new sentence.
            word_list.append(tokenized_text[i]) #Basically, append the first word of the sentence to our array of all words within a sentence.
            
            index = i+1 #This is a counter that lets us loop through the passage until a new sentence is detected
            while tokenized_text[index].is_sent_start == False: #While a new sentence hasn't been detected...
                word_list.append(tokenized_text[index]) #Append the rest of the words in the current sentence of interest to our sentence word list
                if index < len(tokenized_text)-1: #Makes sure we don't have an index error
                    index += 1 #This allows us to move onto the next word
                else: #If the above condition is false, we need to break out of this while loop because we have reached the end of the passage.
                    break;

        ## The code below allows us to calculate the "coherence semantic-similarity" score, or the semantic-similarity score between a target-word and all preceding content words within a sentence.
        prior_sem_sim = [] #This keeps track of all semantic similarity calculations made for any given target word. And resets for each word.
        before_m = word_list[:m] #Basically, this returns of word list of all items that come before the target word at index m
        curr_word = word_list[m] #Save the current word we're looking at -- this is our "target" word.
        for k in range(len(before_m)): #Loop through all words that appeared before the target word at index m
            if not curr_word.is_stop and not before_m[k].is_stop: #I only want to calculate semantic similarity scores between the target word and words that are NOT stop words
                similarity_score = curr_word.similarity(before_m[k]) #Calculate semantic similarity score between target word and primer word
                prior_sem_sim.append(similarity_score) #Append it onto our prior_sem_sim variable, and repeat until there are no more content words that appeared before the target word in the sentence.
        m += 1 #Update m, so that the next loop through, it makes these comparasions with the next word in the word list
        if len(prior_sem_sim) > 0: #Basically checks to make sure that there were actually content words before the target word within the sentence.
            prior_sim_val = sum(prior_sem_sim)/len(prior_sem_sim) #If there are, take the average of all semantic-similarity score in order to get the "coherence semantic similarity" score.
            prior_sem_sim = [] #Reset the value for prior_sem_sim for the next word
        else: #If word is part of a sentence with no content words or just one...
            prior_sim_val = 0 #Assign sem_sim_val to be 0 if the word is a stop word, or a content word with no preceding content word within the same sentence.
            #Any word with a prior_sim_val of 0 will be filtered out during analysis.
        
        if not tokenized_text[i].is_punct and not tokenized_text[i].pos_ == "SPACE": #If the current token is not punctuation or a space
            nw += 1 #Increase the word count by one
            pos_tuple.append((nw,tokenized_text[i].text, prior_sim_val, sing_prior_sim, len(tokenized_text[i].text), sent_num, filename)) #Append variables into a list of tuples

#Finally, use the list of tuples we just created to make a dataframe with all of our words and variable scores
pos_df = pd.DataFrame(pos_tuple, columns =['word_id','word','prior_sim_val', 'single_primer', 'word_len','sent_num','passage'])

pos_df.to_csv(r'pos_Output.csv', index=False) #Export dataframe for sanity check.

pos_df

air.txt
first content word found: air 1
mouse_in_the_house.txt
first content word found: mouse 1
the_brain_and_five_senses.txt
first content word found: people 1
the_surprise.txt
first content word found: sam 1
toads.txt
first content word found: toads 1


C:\Users\tangka\AppData\Local\Temp\ipykernel_11336\3833608638.py:37: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  sing_prior_sim = curr_word.similarity(prior_word) #This calculates the semantic similarity between the "target" word (curr_word) and "primer" word (prior_word)
C:\Users\tangka\AppData\Local\Temp\ipykernel_11336\3833608638.py:66: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_score = curr_word.similarity(before_m[k]) #Calculate semantic similarity score between target word and primer word


,word_id,word,prior_sim_val,single_primer,word_len,sent_num,passage
0,1,air,0.000000,0.000000,3,1,air.txt
1,2,is,0.000000,0.000000,2,1,air.txt
2,3,all,0.000000,0.000000,3,1,air.txt
3,4,around,0.000000,0.000000,6,1,air.txt
4,5,us,0.000000,0.000000,2,1,air.txt
...,...,...,...,...,...,...,...
937,305,toads,0.192754,0.113033,5,27,toads.txt
938,306,in,0.000000,0.000000,2,27,toads.txt
939,307,gardens,0.131438,0.314948,7,27,toads.txt
940,308,and,0.000000,0.000000,3,27,toads.txt


In [21]:
#                                    #
##Importing SUBTLEXus word frequency##
#           (standardized)           #
df_SUBTlog10['Word'] = df_SUBTlog10['Word'].str.lower()
df_SUBTlog10['Word'] = df_SUBTlog10['Word'].str.replace('\W', '', regex=True)

#Sum all duplicates
df_SUBTlog10['TotalLog10'] = df_SUBTlog10.groupby('Word')['Lg10WF'].transform('sum')
df_SUBTlog10['TotalSUBTLWF'] = df_SUBTlog10.groupby('Word')['SUBTLWF'].transform('sum')

#Drop all duplicates
df_ndSUBTlog10 = df_SUBTlog10.drop_duplicates('Word')

#Only take the columns I want
df_ndSUBTlog10 = df_SUBTlog10[['Word','TotalLog10', 'TotalSUBTLWF']]
df_ndSUBTlog10.columns = ['word', 'totallog10', 'totalsubtlwf']

#Merge dfs
pos_df = pos_df.merge(df_ndSUBTlog10, on = 'word', how = 'left')


In [22]:
####                     ####
## MY FUNCTIONS LIVE HERE ###
####                     ####

##REORGANIZING TO FIX WHEN WORDS ARE SPLIT INTO TWO TOKENS##
## ie she'll -> she|'ll ##
## Basically, split the word into two columns, with both columns having the same data ##
## So if a kid had a miscue on "she'll", that represents a miscue on both tokens "she" and "'ll"
## This fix is necessary, because the data with miscue rates have words that are not split in this way
## If this isn't fixed, merging the dataframe with all of our paired and coherence semantic-similarity scores with the dataframe with miscue rate data would not be possible

def apst_sort(text_data):
    index_move_from = []
    index_move_to = []
    
    for (columnName, columnData) in text_data.items():
        #print(columnName) checking to make sure loop ends at original df end-point
        #print(columnData[0])
        
        #Just a backup check to make sure everything being run through nlp() is a string
        if type(columnData[0]) == str:
            nlp_word = nlp(columnData[0])
        
            if len(nlp_word) > 1:
                print(columnData[0])
                word_parts = []
                #duplicate column and append to end of df
                text_data[len(text_data.columns)] = text_data[columnName]

                #This is to fix cases where the word that's being split is due to a - 
                for token in nlp_word:
                    if not token.is_punct and not token.pos_ == "SPACE":
                        word_parts.append(token.text)

                count = 0 #count to keep track of the two word parts
                for part in word_parts:
                    if count == 0:
                        #for the first part, append to where current word is
                        text_data[columnName][0] = part
                        index_move_to.append(columnName) #Keep track of where the current word is
                        count += 1
                    elif count == 1:
                        #Replace word of the newly added column to the second half of word
                        text_data.iloc[0, (len(text_data.columns)-1)] = part
                        index_move_from.append(len(text_data.columns)-1) #Keep track of where the added part of the word is
                        count += 1

    #Now that the new columns are added, we want it to match with the word_ids in our POS_word list
    #This means we need to reorganize the columns to match (we use the columns for word_ids, and since the split words are at the end of the df
    #we need to move them to their rightful place -- right after the word they were split from)

    new_col_names = np.arange(len(text_data.columns)) #this is what the new column names are about to be

    for i in range(len(index_move_to)): #So for however many instances of words being split through tokenization
        #replace the col_name of where the part of the word currently is 
        #with where it SHOULD be (+i because with each loop, the col_names are being shifted)
        new_col_names[index_move_from[i]] = index_move_to[i] + 1 + i 
        #this is a little complicated, basically, only add one to values between where we want to insert our split word, and where the split word currently lives
        new_col_names[(index_move_to[i]+1):(len(new_col_names)-len(index_move_to))] = new_col_names[(index_move_to[i]+1):(len(new_col_names)-len(index_move_to))]+1 

    text_data.columns = new_col_names #now replace the columns with our newly ordered names
    text_data.sort_index(axis=1, inplace=True) #and now let's sort it so that it's back to numerical order (PHEW!)
    return(text_data)

##CREATE REFEREANT TABLE TO ASSIST WITH MERGING POS_DF WITH DATA_DF
def ref_create(text_data, passage):
    #Creating a referant dataframe
    word_index = pd.DataFrame(columns = ['word', 'word_id', 'passage'])
    #Put the words in the word column
    word_index['word'] = text_data.iloc[0].to_frame()
    #Set the column names as the word_id column
    word_index['word_id'] = text_data.columns #np.arange(len(word_index))
    #Set the passages (so that we can later use that information to merge dataframes)
    word_index['passage'] = passage #'toads.csv' ###!!!!! THIS WILL NEED TO BE CHANGED TO FILENAME !!!! ####
    #Get rid of the first row, which is just Word and index 0 (not actually words from the passage)
    word_index = word_index.drop([0]) #Dropping that extra row that had 
                                      #"word" as word, and 0 as index
    #lowercase everything 
    word_index['word'] = word_index['word'].str.lower()
    return(word_index)


In [23]:
############
## This code here converts the csvs that contain
## data on miscue/correct cues into a format that
## can be merged with the pos data-table created above.
##
## Data had to be converted from wide to long using
## the melt function. And fixes had to be made to words
## with an apostrophe in order to succesfully match the tokenized versions of those
## words needed to succesfully merge the two dfs together.
##############

#create a path object that we can loop through to access the text files
reading_dir = Path('data') #using Jupyter Lab - default dir is folder code is in

#create a list of reading files to be looped through later
reading_files = list(reading_dir.glob('*.csv'))

#Create the final df of this done on ALL passages and just stacked on top of each other
merged_stack = pd.DataFrame(columns = ["ID", "word_id", "miscue", "word","passage"])


#How to read multiple file in at once
for file in reading_files:
    csv_data = pd.read_csv(file, dtype=str, header = None)
    csv_name = file.name
    print(csv_name)
    
    csv_data.iloc[0,0] = 'Words'

    ##REORGANIZING TO FIX WHEN WORDS ARE SPLIT INTO TWO TOKENS##
    csv_data = apst_sort(csv_data)

    #Creating a referant dataframe
    word_index = ref_create(csv_data, csv_name)

    #Cleaning the row with just words out of the original df (so that we can melt it)
    csv_data = csv_data.drop(0)

    #This part is kind of strange, but it works, so I won't complain
    #Create a list of all columns as string
    list_str = [str(x) for x in csv_data.columns]
    #Actually name all csv_data columns into stringified indexes (melting doesn't work unless we do this)
    csv_data.columns = list_str

    #Now drop that extra column (because we'll need to use this when melting the df)
    #we don't want value at index 0, because that's our id_vars
    list_str.pop(0)

    #let's melt the data
    csv_data_long = pd.melt(csv_data, id_vars = '0', value_vars = list_str)
    csv_data_long.columns = ['ID', 'word_id', 'miscue'] #renaming columns for merging
    #Turning the word_id column to numpy int32 type, so that merging can happen with word_index
    csv_data_long['word_id'] = csv_data_long['word_id'].astype(np.int32)


    csv_data_merged = csv_data_long.merge(word_index, on = 'word_id', how = 'left')
    
    merged_stack = pd.concat([merged_stack,csv_data_merged])
    
    #csv_data_merged # IT WORKS!!!
    word_index.to_csv('WordList_' + csv_name)

merged_stack

air.csv
can't
mouse_in_the_house.csv
didn't
224-3414
don't
we'll
the_brain_and_five_senses.csv
the_surprise.csv
Sam's
didn't
Sam's
I'll
didn't 
toads.csv
toad's
hunter's


,ID,word_id,miscue,word,passage
0,4140,1,0,air,air.csv
1,4139,1,0,air,air.csv
2,4138,1,0,air,air.csv
3,4137,1,0,air,air.csv
4,4136,1,NaN,air,air.csv
...,...,...,...,...,...
43873,RC_227,309,0,parks,toads.csv
43874,RC_228,309,0,parks,toads.csv
43875,RC_267,309,0,parks,toads.csv
43876,RC_268,309,0,parks,toads.csv


In [24]:
#######
#MORE GENERAL DATA CLEANING
#Removing NAs, white spaces on words, and cleaning dtypes
#for better merge compatability.
#
#Additionally, cleaning passages column:
#"passages.csv/txt" to just be "passage" for
#better merge compatability
#######

#Drop all words that have NA cues (not all participants did every passage)
merged_stack = merged_stack.dropna()

#Remove white spaces improperly added from original csv
merged_stack['word'] = merged_stack['word'].str.strip()

#Converting word_id and miscues to int (they aren't strings)
merged_stack = merged_stack.astype({'word_id': 'int', 'miscue':'int'})

#We want to drop everything after period in the "passage" column"
#We'll be merging based off of matches on word_id, word, and passage, so this is a necessary step
merged_stack[['passage','file_type']] = merged_stack['passage'].str.split(".",expand=True)
pos_df[['passage','file_type']] = pos_df['passage'].str.split(".", expand=True)

C:\Users\tangka\AppData\Local\Temp\ipykernel_11336\1544540147.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_stack['word'] = merged_stack['word'].str.strip()


In [25]:
#Checking to make sure the passage labels match
print(set(merged_stack['passage']) == set(pos_df['passage']))

True


In [26]:
## This merges the pos_data established earlier with the 
## raw miscue/correct cue data that was cleaned and converted
## from wide to long

#Let's drop the unnecessary file_type column
merged_stack = merged_stack.drop(['file_type'], axis=1)
pos_df = pos_df.drop(['file_type'], axis=1)

##Time to merge the data!
final_merge = pd.merge(merged_stack, pos_df, how = 'left', left_on=['word_id','word','passage'], right_on=['word_id','word','passage'])

In [27]:
#Some more data cleaning

#We don't care about what kind of error they made, so let's just make any value that's 1 or above just 1.
final_merge.loc[final_merge['miscue'] > 1, 'miscue'] = 1

#Just a sanity check
#final_merge[final_merge['ID']=='4119'] 


In [28]:
#print(len(pos_df[pos_df['contain_rel'] == 1]),len(pos_df)) 
#Export it (just so I can use a different analysis software later if need be)
final_merge.to_csv('to_be_analyzed.csv')
final_merge

,ID,word_id,miscue,word,passage,prior_sim_val,single_primer,word_len,sent_num,totallog10,totalsubtlwf
0,4140,1,0,air,air,0.000000,0.000000,3,1,3.8507,139.02
1,4139,1,0,air,air,0.000000,0.000000,3,1,3.8507,139.02
2,4138,1,0,air,air,0.000000,0.000000,3,1,3.8507,139.02
3,4137,1,0,air,air,0.000000,0.000000,3,1,3.8507,139.02
4,4133,1,0,air,air,0.000000,0.000000,3,1,3.8507,139.02
...,...,...,...,...,...,...,...,...,...,...,...
85429,RC_227,309,0,parks,toads,0.199556,0.604887,5,27,2.2810,3.73
85430,RC_228,309,0,parks,toads,0.199556,0.604887,5,27,2.2810,3.73
85431,RC_267,309,0,parks,toads,0.199556,0.604887,5,27,2.2810,3.73
85432,RC_268,309,0,parks,toads,0.199556,0.604887,5,27,2.2810,3.73


# Data Analysis

In [30]:
#This allows us to get average miscue rates for each word
grouped_data = final_merge.groupby(by=['word_id', 'word', 'passage','sent_num'], group_keys = True).mean(numeric_only = True)
t_data = grouped_data.reset_index()

t_data.to_csv('t-data.csv') #Converting to csv just so I can double-check
t_data #Data looks good!

#miscue column = average miscue rates across participants on that word
#prior_sim_val = "coherence semantic-similarity" scores, essentially, it's the average semantic-similarity score between the target word and 
                 #all preceding content words that appeared within the same sentence.
#single_primer = "paired semantic-similarity" scores, essentially it's the semantic-similarity score between the target word and 
                 #the closest preceding content word that appeared within the same passage.
#word_len = That's just the word length of a word.
#totallog10 = This is word frequency that has been logorithmicaly transformed.
#totalsubtlwf = This is the raw word frequency (will not be included in analysis, since totallog10 is likely a better (and more linear) measure)

,word_id,word,passage,sent_num,miscue,prior_sim_val,single_primer,word_len,totallog10,totalsubtlwf
0,1,air,air,1,0.169811,0.000000,0.000000,3.0,3.8507,139.02
1,1,all,the_brain_and_five_senses,1,0.075949,0.000000,0.000000,3.0,5.4204,5161.86
2,1,once,mouse_in_the_house,1,0.169811,0.000000,0.000000,4.0,4.2453,344.88
3,1,sam,the_surprise,1,0.096386,0.000000,0.000000,3.0,3.7981,123.16
4,1,toads,toads,1,0.021127,0.000000,0.000000,5.0,1.4914,0.59
...,...,...,...,...,...,...,...,...,...,...
937,305,toads,toads,27,0.035211,0.192754,0.113033,5.0,1.4914,0.59
938,306,in,toads,27,0.035211,0.000000,0.000000,2.0,5.6976,9773.41
939,307,gardens,toads,27,0.056338,0.131438,0.314948,7.0,2.3160,4.04
940,308,and,toads,27,0.042254,0.000000,0.000000,3.0,5.8343,13387.84


In [44]:
#The code below allows us to filter out stop words as well as content words that did not have a preceding content word in the passage/sentence

#The single_primer is our varaible for "paired semantic-similarity" scores. 
t_data = t_data[t_data['single_primer'] != 0] #If it = 0, that means it's either a stop word, or the first content word of a passage.
#We want to exclude that from our analysis, since we didn't make semantic-similarity comparsions for these words.

#The prior_sim_val is our variable for "coherence semantic-similarity" scores.
t_data_prior = t_data[t_data['prior_sim_val']!=0] #If it = 0, that means it's either a stop word, or the first content word of a sentence.
#We want to exclude that from our analysis, since we didn't make semantic-similarity comparsions for these words.

#Export it for further analysis in R.
t_data.to_csv("single_R_Data.csv")
t_data_prior.to_csv("prior_R_Data.csv")

In [45]:
#This code takes a chunk of t_data (just our variables of interest) to be used for calculating correlations.
spruced_data = t_data_prior[['miscue', 'prior_sim_val', 'single_primer', 'word_len', 'totallog10']]

In [46]:
spruced_data

,miscue,prior_sim_val,single_primer,word_len,totallog10
12,0.084337,0.081770,0.081770,8.0,3.6954
18,0.007042,0.194954,0.194954,5.0,4.3053
23,0.126582,0.403196,0.403196,6.0,2.6365
30,0.060241,0.077003,0.206933,4.0,4.1929
36,0.075949,0.206220,0.206220,4.0,4.0531
...,...,...,...,...,...
932,0.000000,0.300969,0.300969,6.0,4.3589
935,0.021127,0.135396,0.173675,3.0,3.4847
937,0.035211,0.192754,0.113033,5.0,1.4914
939,0.056338,0.131438,0.314948,7.0,2.3160


In [47]:
matrix = spruced_data.corr()
print(matrix)

                 miscue  prior_sim_val  single_primer  word_len  totallog10
miscue         1.000000      -0.086004      -0.087655  0.072306   -0.103650
prior_sim_val -0.086004       1.000000       0.773533  0.290319   -0.123401
single_primer -0.087655       0.773533       1.000000  0.226175   -0.097613
word_len       0.072306       0.290319       0.226175  1.000000   -0.362521
totallog10    -0.103650      -0.123401      -0.097613 -0.362521    1.000000


Single primer and prior_sim_val are highly correlated (to be expected)

**Word length and the logorithmic word frequency are NOT (surprising!)**
  * Could be a few reasons of this. For one, since these are texts made for kids, I'm guessing word frequency values would be quite high for most words. This means that irregardless of word length, word frequency could be similar. Though, I dont' think there's anything I can do to confirm this, other than maybe looking at a hsitogram to see the distribution of word frequency for words used in this anlaysis.